<h3 style="color:red;">---------- Make sure the Docker service running before running this notebook ----------</h3>

# Section 1: Launching HaGrid nodes

In [1]:
! hagrid launch {"LOCAL_NETWORK"} network to docker:8000 --build-src=0.7.0

✅ Updated HAGrid from branch: 0.7.0m
⠴ Updating HAGrid from branch: 0.7.0
⠼ Checking for Docker Service   ice   
✅ Docker service is running
✅ Git 2.37.1
✅ Docker 20.10.22
✅ Docker Compose 2.15.1


                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNmdmNNMMNNNy/oso

In [2]:
from utils import *

local_network = sy.login("localhost", 8000)

Connecting to localhost... done! 	 Logging into local_network... as GUEST...done!


<h3 style="color:red;">---------- Run Sections 1 in Domain_1 and Domain_2 notebooks ----------</h3>

In [4]:
local_network.domains

,host_or_ip,id,is_vpn,name,port,private,protocol
0,100.64.0.2,6f82b41b82c145c58c966a581a65e585,1,domain_1,80,0,http
1,100.64.0.3,dbe40ab327af41cdae501872adbc193e,1,domain_2,80,0,http


---

# Section 2 : Train the dataset and Upload Weights

Generally, when you have to deal with image, text, audio or video data,
you can use standard python packages that load data into a numpy array.
Then you can convert this array into a ``torch.*Tensor``.

-  For images, packages such as Pillow, OpenCV are useful
-  For audio, packages such as scipy and librosa
-  For text, either raw Python or Cython based loading, or NLTK and
   SpaCy are useful

Specifically for vision, we have created a package called
``torchvision``, that has data loaders for common datasets such as
ImageNet, CIFAR10, MNIST, etc. and data transformers for images, viz.,
``torchvision.datasets`` and ``torch.utils.data.DataLoader``.

This provides a huge convenience and avoids writing boilerplate code.


### 2. 0. Training an image classifier

We will do the following steps in order:

1. Load and normalize the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

#### 2. 1. Load and normalize CIFAR10

Using ``torchvision``, it’s extremely easy to load CIFAR10.

The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].

Federated learning can be applied to many different types of tasks across different domains. In this tutorial, we introduce federated learning by training a simple convolutional neural network (CNN) on the popular CIFAR-10 dataset. CIFAR-10 can be used to train image classifiers that distinguish between images from ten different classes:

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original CIFAR-10 dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes, in the real world there's no need for data splitting because each organization already has their own data (so the data is naturally partitioned).

Each organization will act as a client in the federated learning system. So having ten organizations participate in a federation means having ten clients connected to the federated learning server:



Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap the resulting partitions by creating a PyTorch `DataLoader` for each of them:

In [5]:
num_clients = 10
manual_seed = 100
trainloaders, valloaders, testloaders = load_datasets(num_clients, manual_seed)

print("\nNumber of train images in domain 2:", len(trainloaders[0]))
print("\nNumber of val images in domain 2:", len(valloaders[0]))
print("\nNumber of test images in domain 2:", len(testloaders[0]))

Files already downloaded and verified
Files already downloaded and verified

Number of train images in domain 2: 141

Number of val images in domain 2: 16

Number of test images in domain 2: 32


The output above shows a random batch of images from the first `trainloader` in our list of ten `trainloaders`. It also prints the labels associated with each image (i.e., one of the ten possible labels we've seen above). If you run the cell again, you should see another batch of images.

#### 2. 2. Define a Convolutional Neural Network

In [6]:
network_model = Net()

for param in network_model.parameters():
    print(param.data.shape)


torch.Size([6, 3, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


#### 2. 3. Define a Loss function and optimizer
Let's use a Classification Cross-Entropy loss and SGD with momentum.

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network_model.parameters(), lr=0.001, momentum=0.9)

#### 2. 4. Training the model

We now have all the basic building blocks we need: a dataset, a model, a training function, and a test function. Let's put them together to train the model on the dataset of one of our organizations (`trainloaders[0]`). This simulates the reality of most machine learning projects today: each organization has their own data and trains models only on this internal data: 

In [8]:
for epoch in range(1, 11):
    train(network_model, trainloaders[0], epoch)
    loss, accuracy = test(network_model, valloaders[0])
    
    loss = round(loss, 2)
    accuracy = round(accuracy, 2)
    
    print(f"Epoch {epoch}: Validation Loss: {loss}, Accuracy: {accuracy*100}%")

Epoch 1: Validation Loss: 0.06, Accuracy: 28.999999999999996%
Epoch 2: Validation Loss: 0.06, Accuracy: 38.0%
Epoch 3: Validation Loss: 0.05, Accuracy: 44.0%
Epoch 4: Validation Loss: 0.05, Accuracy: 47.0%
Epoch 5: Validation Loss: 0.05, Accuracy: 48.0%
Epoch 6: Validation Loss: 0.07, Accuracy: 47.0%
Epoch 7: Validation Loss: 0.12, Accuracy: 46.0%
Epoch 8: Validation Loss: 0.16, Accuracy: 46.0%
Epoch 9: Validation Loss: 0.19, Accuracy: 40.0%
Epoch 10: Validation Loss: 0.21, Accuracy: 42.0%


#### 2. 5. Test the network on the test data

We have trained the network for 2 passes over the training dataset.
But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

In [9]:
loss, accuracy = test(network_model, testloaders[0])
print(f"\nFinal test set performance:\n\tLoss {round(loss, 2)}\n\tAccuracy: {round(accuracy*100, 2)}%")


Final test set performance:
	Loss 0.18
	Accuracy: 43.0%


---

#### 2. 5. Annotate Data for Automatic DP

Now we need to annotate our data with some Auto DP `(Automatic Differential Privacy)` metadata.

In order to protect the `privacy` of the people within our dataset we first need to specify who those people are. 

To help obscure the variables someone may learn about these datasets we then need to set an appropriate `lower_bound`, and `upper_bound`.

Once you have prepared your data, it’s time to upload it to your Domain Server.  To help Data Scientists later `search` and `discover` our datasets, we will add details like a `name` and a `description` of what this dataset represents.

In [14]:
domain1 = sy.login(email="info@openmined.org", password="changethis", port=8001)

domain1.datasets


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into domain_1... done!


In [15]:
domain2 = sy.login(email="info@openmined.org", password="changethis", port=8002)

domain2.datasets


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into domain_2... done!


In [16]:
i, j = 0, 0
for layer in network_model.children():
        
    if "weight" in layer.state_dict():
        i += 1
            
        model_weights = layer.state_dict()["weight"]
        weights_tensor = sy.Tensor(model_weights).annotate_with_dp_metadata(lower_bound=-1, upper_bound=1, data_subject="PyTorch Weights")

        domain1.load_dataset(assets={"Weight {}".format(i): weights_tensor},
                              name="Initial Weight {}".format(i),
                              description="Initial PyTorch weights sent by Network.")

        domain2.load_dataset(assets={"Weight {}".format(i): weights_tensor},
                              name="Initial Weight {}".format(i),
                              description="Initial PyTorch weights sent by Network.")

    if "bias" in layer.state_dict():
        j += 1

        model_bias = layer.state_dict()["bias"]
        bias_tensor = sy.Tensor(model_bias).annotate_with_dp_metadata(lower_bound=-1, upper_bound=1, data_subject="PyTorch Bias")

        domain1.load_dataset(assets={"Bias {}".format(j): bias_tensor},
                              name="Initial Bias {}".format(j),
                              description="Initial PyTorch biases sent by Network.")

        domain2.load_dataset(assets={"Bias {}".format(j): bias_tensor},
                              name="Initial Bias {}".format(j),
                              description="Initial PyTorch biases sent by Network.")
        

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /Users/neeleshkarthikeyan


Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Weight 1`: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 81.64it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Loading dataset... uploading...🚀                        


Uploading `Weight 1`: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 150.02it/s]


...

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Bias 5`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 184.27it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Loading dataset... uploading...🚀                        

Uploading `Bias 5`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 177.85it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [17]:
domain1.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",6a7d9a50-768d-4b50-91c3-7afc378c3d44
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",863b2e56-f7ac-4c65-a6f6-cc1773d5df7f
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",2d8b86bf-6136-45be-a55e-b32f745b4d75
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",165b16f5-7d38-4e06-abac-ff8ac19538bb
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",73ba01b9-b2d7-4cfd-a2b2-78018420fcfa
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",712206ef-e926-4e5a-945a-2dfa0221bb9f
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",56bb309d-80b1-4197-ace7-d8ef1b123f6a
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",c553b5cd-6f5a-4a97-8f75-11a7df37e413
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",aebe0022-575e-4711-b5c6-40ce78072970
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",4be73095-2d88-4cee-851a-6e944927d2ed


In [18]:
domain2.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",5d4aed9b-2441-468d-bc06-cfb2f0fd61dc
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",209472e3-c907-432c-b8f7-f91076a31e70
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",3152fe92-039b-43f2-87d6-e01cad5151d0
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",262d9293-f797-4678-b13f-7b460b30546e
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",4ec8e1d5-484b-40c5-b40b-5c37e5815e3b
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",16296dd1-9080-4ae8-b357-04509720cf76
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",99758131-00f4-481c-8c05-ceee256c502c
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",91e81bc5-56db-4748-b651-ce8884d66ad0
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",1541bb33-a494-4d3a-9c75-4dac1f72157e
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",42293406-6359-48b2-a0f1-0f97f97afb19


---

# Section 3: Aggregate the Model Weights

<h3 style="color:red;">---------- Run the below cells after running Section 2 in Domain_1 and Domain_2 notebooks ----------</h3>

In [19]:
j = 1

for i in range(10, 20):
    
    if i % 2 == 0:
        weight_1 = domain1.datasets[i]["Weight {}".format(j)].get().child.value
        weight_2 = domain2.datasets[i]["Weight {}".format(j)].get().child.value
        mean_weights = torch.FloatTensor(np.mean(np.array([weight_1, weight_2]), axis=0))
        
        weights_tensor = sy.Tensor(mean_weights).annotate_with_dp_metadata(lower_bound=-1, 
                                                                           upper_bound=1, 
                                                                           data_subject="PyTorch Weights")
        
        domain1.load_dataset(assets={"Weight {}".format(j): weights_tensor},
                              name="Final Weight {}".format(j),
                              description="Final PyTorch weights sent by Network.")

        domain2.load_dataset(assets={"Weight {}".format(j): weights_tensor},
                              name="Final Weight {}".format(j),
                              description="Final PyTorch weights sent by Network.")

            
    else:
        bias_1 = domain1.datasets[i]["Bias {}".format(j)].get().child.value
        bias_2 = domain1.datasets[i]["Bias {}".format(j)].get().child.value
        mean_bias = torch.FloatTensor(np.mean(np.array([bias_1, bias_2]), axis=0))
        
        bias_tensor = sy.Tensor(mean_bias).annotate_with_dp_metadata(lower_bound=-1, 
                                                                     upper_bound=1, 
                                                                     data_subject="PyTorch Bias")

        domain1.load_dataset(assets={"Bias {}".format(j): bias_tensor},
                              name="Final Bias {}".format(j),
                              description="Final PyTorch biases sent by Network.")

        domain2.load_dataset(assets={"Bias {}".format(j): bias_tensor},
                              name="Final Bias {}".format(j),
                              description="Final PyTorch biases sent by Network.")
        
        j += 1
    
    

Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Weight 1`: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 158.48it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Loading dataset... uploading...🚀                        

Uploading `Weight 1`: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 121.94it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                                                                                                                                             

...

Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Bias 5`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 148.90it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Loading dataset... uploading...🚀                        

Uploading `Bias 5`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 164.83it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [20]:
domain1.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",6a7d9a50-768d-4b50-91c3-7afc378c3d44
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",863b2e56-f7ac-4c65-a6f6-cc1773d5df7f
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",2d8b86bf-6136-45be-a55e-b32f745b4d75
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",165b16f5-7d38-4e06-abac-ff8ac19538bb
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",73ba01b9-b2d7-4cfd-a2b2-78018420fcfa
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",712206ef-e926-4e5a-945a-2dfa0221bb9f
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",56bb309d-80b1-4197-ace7-d8ef1b123f6a
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",c553b5cd-6f5a-4a97-8f75-11a7df37e413
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",aebe0022-575e-4711-b5c6-40ce78072970
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",4be73095-2d88-4cee-851a-6e944927d2ed


In [21]:
domain2.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",5d4aed9b-2441-468d-bc06-cfb2f0fd61dc
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",209472e3-c907-432c-b8f7-f91076a31e70
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",3152fe92-039b-43f2-87d6-e01cad5151d0
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",262d9293-f797-4678-b13f-7b460b30546e
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",4ec8e1d5-484b-40c5-b40b-5c37e5815e3b
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",16296dd1-9080-4ae8-b357-04509720cf76
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",99758131-00f4-481c-8c05-ceee256c502c
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",91e81bc5-56db-4748-b651-ce8884d66ad0
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",1541bb33-a494-4d3a-9c75-4dac1f72157e
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",42293406-6359-48b2-a0f1-0f97f97afb19


<h3 style="color:red;">--------------- Run Section 3 in Domain_1 and Domain_2 notebooks ---------------</h3>